In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import os
import yaml
current_path = os.getcwd()
print(current_path)
assert current_path.split('/')[-1]=="mastodon-sim", "run the notebook from project root!"

/mnt/c/Users/maxpu/Dropbox/scripts/Projects/socialsandbox/mastodon-sim


load

In [4]:
PROJECT_ROOT = current_path
print("project root: " + str(PROJECT_ROOT))
abs_rootpath = PROJECT_ROOT + "/examples/election/outputs"
print(abs_rootpath)

project root: /mnt/c/Users/maxpu/Dropbox/scripts/Projects/socialsandbox/mastodon-sim
/mnt/c/Users/maxpu/Dropbox/scripts/Projects/socialsandbox/mastodon-sim/examples/election/outputs


In [5]:
# in output_proc_utils.py

import collections

def load_data(fileroot, names_of_focalplayers):
    with open(fileroot[0]+'/'+fileroot[1] +"/.hydra/config.yaml") as file:
        config_data = yaml.safe_load(file)
    outfile_path = fileroot[0]+'/'+fileroot[1]+'/'+fileroot[1]+'_'+fileroot[2] + "_events.jsonl"
    print(outfile_path)
    df = pd.read_json(outfile_path, lines=True)

    pd.set_option("display.width", 1000)
    print(df.head())
    print()
    print("Probes:")
    print(df.loc[df.event_type == "probe", "label"].value_counts())
    print()
    print("Actions:")
    print(df.loc[(df.event_type == "action") & (df.episode > -1), "label"].value_counts())
    print()
    print(
        df.loc[(df.event_type == "action") & (df.episode > -1) & df.data.apply(lambda x: type(x)==dict), "data"]
        .apply(lambda x: x.keys())
        .value_counts()
    )
    print()
    print(
        df.loc[
            (df.source_user.isin(names_of_focalplayers))
            & (df.event_type == "action")
            & (df.episode > -1)
        ]
        .groupby("source_user")["label"]
        .value_counts()
    )
    print()
    print("post")
    posts = df.loc[
        (df.label == "post") & (df.episode > -1) & (df.source_user != "storhampton_gazette"),
        ["source_user", "data"],
    ]
    users = posts["source_user"].values
    posts = posts["data"].apply(lambda x: x["post_text"]).values
    for user, post in zip(users, posts):
        print(f"source_user:{user}:{post}")
    print()
    print("reply")
    replies = df.loc[
        (df.label == "reply") & (df.episode > -1) & (df.source_user != "storhampton_gazette"),
        ["source_user", "episode", "data"],
    ]
    users = replies["source_user"].values
    replies = replies["data"].apply(lambda x: x["post_text"]).values
    for user, reply in zip(users, replies):
        print(f"source_user:{user}:{reply}")
    print()
    dftmp = df.copy()
    dftmp = dftmp.loc[(dftmp.event_type == "action") & (dftmp.episode > -1), :]
    dftmp["data"] = dftmp["data"].apply(str)
    print(f"{(len(dftmp) - len(dftmp.drop_duplicates())) / len(dftmp):.3f} duplicate fraction")
    return df


def post_process_output(df):
    eval_df = df.loc[
        df.event_type == "eval", ["episode", "source_user", "label", "data"]
    ].reset_index(drop=True)
    eval_df["response"] = eval_df.data.apply(lambda x: x["query_return"])
    eval_df = eval_df.drop("data", axis=1)

    edge_df = df.loc[
        df.label.isin(["follow", "unfollow"]), ["episode", "source_user", "data", "label"]
    ].reset_index(drop=True)
    edge_df["target_user"] = edge_df.data.apply(lambda d: d["target_user"])
    edge_df = edge_df.drop("data", axis=1)

    interaction_types = ["post", "like_toot", "boost_toot", "reply"]
    int_df = df.loc[df.label.isin(interaction_types), :].reset_index(drop=True)
    return eval_df, int_df, edge_df


def episodewise_graphbuild(edge_df):
    follow_graph = nx.DiGraph()
    for epi_edge_data in edge_df.groupby("episode"):
        for action, operate_on_graph in zip(
            ["follow", "unfollow"], [follow_graph.add_edges_from, follow_graph.remove_edges_from]
        ):
            if (epi_edge_data.label == action).any():
                data = epi_edge_data.loc[
                    epi_edge_data.label == action, ["source_user", "target_user"]
                ]
                operate_on_graph(list(data.itertuples(index=False, name=None)))
    return follow_graph


# in dashboard-basic.py:


def get_target_user(row):
    if row.label == "post":
        target_user = row.source_user
    elif row.label == "like_toot":
        target_user = row.data["target_user"]
    elif row.label == "boost_toot":
        target_user = row.data["target_user"]
    elif row.label == "reply":
        target_user = row.data["reply_to"]["target_user"]
    return target_user


def get_int_dict(int_df):
    past = dict(
        zip(["post", "like_toot", "boost_toot", "reply"], ["posted", "liked", "boosted", "replied"])
    )
    int_df["int_data"] = int_df.apply(
        lambda x: {
            "action": past[x.label],
            "episode": x.episode,
            "source_user": x.source_user,
            "target_user": get_target_user(x),
            "toot_id": x.data["toot_id"],
        },
        axis=1,
    )
    int_df.int_data = int_df.apply(
        lambda x: x.int_data | {"parent_toot_id": x.data["reply_to"]["toot_id"]}
        if x.label == "reply"
        else x.int_data,
        axis=1,
    )
    return int_df.groupby("episode")["int_data"].apply(list).to_dict()


def get_toot_dict(int_df):
    past = dict(
        zip(["post", "like_toot", "boost_toot", "reply"], ["posted", "liked", "boosted", "replied"])
    )
    text_df = int_df.loc[(int_df.label == "post") | (int_df.label == "reply"), :].reset_index(
        drop=True
    )

    # handle Nones as toot_ids by appending an index
    no_toot_id = text_df.data.apply(lambda x: x["toot_id"] is None)
    text_df["no_toot_id_idx"] = -1
    text_df.loc[no_toot_id, "no_toot_id_idx"] = range(no_toot_id.sum())
    text_df.loc[no_toot_id, "data"] = text_df.loc[no_toot_id, :].apply(
        lambda x: x.data | {"toot_id": "None" + str(x.no_toot_id_idx)}, axis=1
    )

    text_df["toot_id"] = text_df.data.apply(lambda x: x["toot_id"])
    text_df = text_df.set_index("toot_id")
    text_df["text_data"] = text_df.apply(
        lambda x: {"user": x.source_user, "action": past[x.label], "content": x.data["post_text"]},
        axis=1,
    )
    text_df.text_data = text_df.apply(
        lambda x: x.text_data | {"parent_toot_id": x.data["reply_to"]["toot_id"]}
        if x.label == "reply"
        else x.text_data,
        axis=1,
    )

    return text_df.text_data.to_dict()


def load_data_dash(eval_df, int_df, edge_df):
    # votes
    votes = (
        eval_df.loc[eval_df.label == "vote_pref", ["source_user", "response", "episode"]]
        .groupby("episode")
        .apply(lambda x: dict(zip(x.source_user, x.response)))
        .to_dict()
    )

    # final follow network
    follow_graph = nx.from_pandas_edgelist(
        edge_df, "source_user", "target_user", create_using=nx.DiGraph()
    )  # invalid in presence of unfollows (in which case use episodewise_graphbuild)

    # active users with episode keys
    posted_users_by_episode = int_df.groupby("episode")["source_user"].apply(set).to_dict()

    # interaction data
    int_dict = get_int_dict(int_df.copy())

    # toot_data
    toot_dict = get_toot_dict(int_df.copy())

    return follow_graph, int_dict, posted_users_by_episode, toot_dict, votes

In [6]:
run_name = ["N20_T8_Reddit.Big5_independent_v1_news_no_bias_with_images_run1","2025-03-22_02-26-35"]
run_name = ["N20_T4_Reddit.Big5_independent_v1_news_no_bias_with_images_run1","2025-04-09_23-34-04"]



names_of_focalplayers = ["Bill Fredrickson", "Bradley Carter"]
df = load_data([abs_rootpath] + run_name, names_of_focalplayers)


/mnt/c/Users/maxpu/Dropbox/scripts/Projects/socialsandbox/mastodon-sim/examples/election/outputs/N20_T4_Reddit.Big5_independent_v1_news_no_bias_with_images_run1/N20_T4_Reddit.Big5_independent_v1_news_no_bias_with_images_run1_2025-04-09_23-34-04_events.jsonl
      source_user   label                                               data  episode event_type
0  Michael Harris  follow  {'target_user': 'Rahul Desai', 'suggested_acti...       -1     action
1      Emily Chen  follow  {'target_user': 'Emily Chen', 'suggested_actio...       -1     action
2    Lisa Collins  follow  {'target_user': 'Carlos Mendes', 'suggested_ac...       -1     action
3    Jason Miller  follow  {'target_user': 'Emily Chen', 'suggested_actio...       -1     action
4  Michael Harris  follow  {'target_user': 'Jessica Rodriguez', 'suggeste...       -1     action

Probes:
label
Favorability    160
VotePref         80
VoteIntent       80
Name: count, dtype: int64

Actions:
label
inner_actions       313
reply              

In [ ]:
df[df.label=='inner_actions'].groupby(['source_user', 'episode'])['data'].apply(lambda x: x.str.count("SUCCESS")).mean()

0.6485623003194888

In [24]:
df[df.label=='inner_actions'].groupby(['source_user', 'episode'])['data'].apply(lambda x: ''.join([('0' if act[:7]=='FAILED ' else '1') for act in x ])).unstack()

episode,0,1,2,3
source_user,,,,
Alex Thompson,110,110,NaN,11
Bill Fredrickson,NaN,NaN,NaN,111
Bradley Carter,NaN,NaN,NaN,110010
Carlos Mendes,11000,11,1100000001,NaN
Chris Anderson,1111111,1101111111011,1110101,11001
Emily Chen,11,11,11,11
Emily Thompson,11,11,NaN,111
Jason Miller,11000000000000000011,1100000000011,110110000010,1100000000010
Jessica Rodriguez,111,11111,11,110001


In [ ]:
import seaborn as sns
import matplotlib.pyplot as pl

#copy from voter model file
action_probabilities = {
    # High frequency actions
    "like_toot": 0.35,  # Most common action
    "boost_toot": 0.15,  # Common but less than likes
    "post": 0.20,  # Regular posting
    "reply": 0.15,
    # Medium frequency actions
    "follow": 0.15,  # Following new accounts
    "unfollow": 0.00,  # 25,  # Unfollowing accounts
    "print_timeline": 0.0,  # Reading timeline
    # Low frequency actions
    "block_user": 0.0,  # Blocking problematic users
    "unblock_user": 0.0,  # Unblocking users
    "delete_posts": 0.0,  # Deleting own posts
    "update_bio": 0.0,  # Updating profile
    "print_notifications": 0.00,  # 25,  # Checking notifications
}

fig, ax = pl.subplots()
dft = df.loc[
    (df.event_type == "action") & (df.episode > -1) & ~(df.source_user.isin(names_of_focalplayers)),
    :,
].copy()
dft["suggested_action"] = dft["data"].apply(
    lambda x: "post" if x["suggested_action"] == "toot" else x["suggested_action"]
)
sns.histplot(
    data=dft.loc[:, ["label", "suggested_action"]].melt(),
    ax=ax,
    x="value",
    hue="variable",
    multiple="dodge",
    shrink=0.75,
    bins=20,
)
dft
action_list = [obj.get_text() for obj in ax.get_xticklabels()]
ax.plot(
    range(len(action_list)),
    [action_probabilities[action] * len(dft) for action in action_list],
    ".-",
)
ax.set_title("line is expected count from suggested freqs")
fig, ax = pl.subplots()
contingency = pd.crosstab(dft["suggested_action"], df["label"])
sns.heatmap(
    contingency,
    ax=ax,
    annot=True,  # Show numbers in cells
    fmt="d",  # Format as integers
    cmap="YlOrRd",  # Yellow-Orange-Red color scheme
    cbar_kws={"label": "Count"},
    square=True,
)  # Make cells square
ax.set_ylabel("action taken")
ax.set_ylabel("suggested action")
ax.tight_layout()